# Set up

In [ ]:
using IntervalArithmetic, Combinatorics, Polynomials, Serialization, LinearAlgebra, Plots, LaTeXStrings

In [ ]:
include("quadrature.jl")

In [ ]:
# start here with precomputed vandermonde matrices
n = big(500)
setprecision(256)

# approximate solution
ū = interval.(deserialize("ubar"));

# regularised Vandermonde matrices for 2p and p+1 products
V̄4 = deserialize("V4r")
V̄6 = deserialize("V6r")

p = interval(3)
d = 2
α = d//2-1

# m = n+1, (n+1)ᵗʰ eigenvalue
λₘ = interval(d//2+n+1)
λ₀ = interval(d//2)
Z = interval(4)*interval(BigFloat, π)

In [ ]:
𝔏 = Diagonal(interval.(big.(collect(0:n))).+λ₀);

In [ ]:
LinearAlgebra.norm(v::Vector) = sqrt(sum(v.^2))

# computes Lᵖ/H² norms before taking fractional exponent
function L2(u)
    return sum(u.^2)
end

function H2(u)
    return sum((𝔏*u.^2))
end

function L6(u)
    return sum((V̄6*u).^6)
end

# rigorous upper bound of the 2-norm of a matrix
function op_norm(A)
    if size(A) == (2,2)
        Z = sqrt(sum(A.^2) + sqrt(((A[1,2]+A[2,1])^2+(A[1,1]-A[2,2])^2)*((A[1,2]-A[2,1])^2+(A[1,1]+A[2,2])^2)))/sqrt(interval(2))
        if isguaranteed(Z)
            return interval(sup(Z))
        else
            return Z
        end
    else
        all(isguaranteed.(A)) || error("matrix not guaranteed")
        return sqrt(interval(maximum(sup.(sum(abs.(A), dims = 1))))*interval(maximum(sup.(sum(abs.(A), dims = 2)))))
    end
end

# Proof

In [ ]:
# rigorous L²-norms of ūᵖ⁻¹ψ̂ⱼ
int = sum(((V̄6*ū).^(p-interval(1)).*V̄6).^2, dims = 1);

In [ ]:
Ḡ = interval.(Float64, V̄4)'*(interval.(Float64, V̄4*ū).^(p-interval(1)) .*interval.(Float64, V̄4));

In [ ]:
# PₙDF(ū)Pₙ
DFū = interval.(Float64, interval(I(n+1)) - inv(𝔏)/(p-interval(1)) - p*𝔏\Ḡ);

In [ ]:
# approximate numerical inverse of PₙDF(ū)Pₙ
Aₙ = interval.(inv(mid.(DFū)));

In [ ]:
w = sqrt.(abs.([int[i] - L2(Ḡ[i,:]) for i=1:n+1]));

In [ ]:
# PₙF(ū)
PFū = ū - inv(𝔏)*(ū/(p-interval(1))+(V̄4'*((V̄4*ū).^p)));

In [ ]:
Y = interval.(sqrt(H2(Aₙ*PFū)+(L6(ū)-L2((V̄4'*((V̄4*ū).^p))))))

In [ ]:
Z¹² = p*norm(abs.(𝔏*Aₙ*inv(𝔏))*w)/λₘ;

In [ ]:
supū = sum(abs.(ū))

In [ ]:
Z¹¹ = op_norm(𝔏*(interval(I(n+1)) - Aₙ*DFū)*inv(𝔏));
Z²¹ = p*norm(𝔏\w);
Z²² = (interval(1)/(p-interval(1))+p*supū)/λₘ;

In [ ]:
Z₁ = op_norm([Z¹¹ Z¹² ; Z²¹ Z²²])

In [ ]:
if sup(Z₁)>=1
    println("ū cannot be validated")
end

In [ ]:
[Z¹¹ Z¹² ; Z²¹ Z²²]

In [ ]:
# Calculation of the L²-norm of the Hessian of η and of the sup of η
C₀ = I"0.56419"
C₁ = I"0.79789"
C₂ = I"0.23033"
# C(2)
C2 = sqrt(Z)*(interval(2//d)*C₀ + sqrt(interval(2//d))*C₁ + C₂)

In [ ]:
op_n = interval(max(sup(op_norm(𝔏*Aₙ*inv(𝔏))), 1))
Z₂ = interval(6)*C2*supū*op_n
Z₃ = interval(6)*C2^2*op_n

In [ ]:
δ̄ = (-Z₂+sqrt(Z₂^2 +interval(2)*Z₃-interval(2)*Z₁*Z₃))/Z₃

In [ ]:
P(δ) = Z₃/interval(6)*δ^3 + Z₂/interval(2)*δ^2 - (interval(1) - Z₁)*δ + Y

In [ ]:
δ̲ = Y/(interval(1)-Z₁)*(interval(1)+interval(BigFloat,2)^(-33))
if sup(P(δ̲))<0
    println("δ̲ validated")
end

# Checking Positivity

In [ ]:
# infinity norm error
inf_err = C2*δ̲

In [ ]:
m = 10
supm = inf_err+sum(abs.(ū[m+2:end]))
Pₘ = zeros(Interval{BigFloat},m+1)
for j = 0:m
#     println(j)
    Lⱼ =  [interval((-1)^i)*interval(BigFloat, binomial(j,i)//factorial(i)) for i=0:j]
    Pₘ[1:j+1] += ū[Int64(j+1)]*Lⱼ
end

In [ ]:
r₀ = interval(BigFloat,4.0)
z = mince(interval(0,r₀.^2/interval(4)), 2000)

if all(inf.(evalpoly.(z,tuple(Pₘ)).*exp.(-z).-supm.*exp.(-z/interval(2))).>0) && inf(interval(d//4)-interval(1)/(p-interval(1))+r₀^2/interval(16)-((supū+inf_err)*exp(-r₀^2/interval(8)))^(p-interval(1)))>0
    println("positivity checked")
else
    println("positivity not checked")
end

# Plotting

In [ ]:
#solution plotting

rloc = collect(0:400)/100
zloc = rloc.^2/4
sol = zeros(size(zloc));
for k = 0:big(n)
    # println(k)
    Lₖ = Polynomial([(-1)^i*big(binomial(k,i)//factorial(i)) for i=0:k])
    sol .+= Float64.(mid.(ū[Int64(k+1)]) .* Lₖ.(zloc))
end

In [ ]:
plot(rloc, sol.*exp.(-zloc), xlabel = L"r", ylabel = L"\bar{u}", legend = false, dpi =1000)

In [ ]:
png("semi2dsol")